# MIRA: Final Project 2023
Professor: Josep Quintana, Robert Martí

Topic: Image registration of chest CT volumes: 4DCT DIR-Lab Challenge

Taiabur


Python script that reads raw images and converts them into NIfTI format. The code using the SimpleITK (sitk) library for medical image processing

In [70]:
import argparse
import tempfile
import SimpleITK as sitk
import os
import matplotlib.pyplot as plt
import pandas as pd
import glob
import ast

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  
import nibabel as nib
from pathlib import Path
from utils.timer_util import Timer

In [71]:

def read_raw(
    binary_file_name,
    image_size,
    sitk_pixel_type,
    image_spacing=None,
    image_origin=None,
    big_endian=False,
):
    pixel_dict = {
        sitk.sitkUInt8: "MET_UCHAR",
        sitk.sitkInt8: "MET_CHAR",
        sitk.sitkUInt16: "MET_USHORT",
        sitk.sitkInt16: "MET_SHORT",
        sitk.sitkUInt32: "MET_UINT",
        sitk.sitkInt32: "MET_INT",
        sitk.sitkUInt64: "MET_ULONG_LONG",
        sitk.sitkInt64: "MET_LONG_LONG",
        sitk.sitkFloat32: "MET_FLOAT",
        sitk.sitkFloat64: "MET_DOUBLE",
    }
    direction_cosine = [
        "1 0 0 1",
        "1 0 0 0 1 0 0 0 1",
        "1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1",
    ]
    dim = len(image_size)
    header = [
        "ObjectType = Image\n".encode(),
        (f"NDims = {dim}\n").encode(),
        (
            "DimSize = " + " ".join([str(v) for v in image_size]) + "\n"
        ).encode(),
        (
            "ElementSpacing = "
            + (
                " ".join([str(v) for v in image_spacing])
                if image_spacing
                else " ".join(["1"] * dim)
            )
            + "\n"
        ).encode(),
        (
            "Offset = "
            + (
                " ".join([str(v) for v in image_origin])
                if image_origin
                else " ".join(["0"] * dim) + "\n"
            )
        ).encode(),
        ("TransformMatrix = " + direction_cosine[dim - 2] + "\n").encode(),
        ("ElementType = " + pixel_dict[sitk_pixel_type] + "\n").encode(),
        "BinaryData = True\n".encode(),
        ("BinaryDataByteOrderMSB = " + str(big_endian) + "\n").encode(),
        (
            "ElementDataFile = " + os.path.abspath(binary_file_name) + "\n"
        ).encode(),
    ]
    fp = tempfile.NamedTemporaryFile(suffix=".mhd", delete=False)

    print(header)

    # Not using the tempfile with a context manager and auto-delete
    # because on windows we can't open the file a second time for ReadImage.
    fp.writelines(header)
    fp.close()
    img = sitk.ReadImage(fp.name)
    os.remove(fp.name)
    
    return img

def convert_raw_to_nifti(raw_file_path, image_size, sitk_pixel_type, output_path,voxel_spacing):
    # Read the raw image

    image = read_raw(
        binary_file_name=raw_file_path,
        image_size=image_size,
        sitk_pixel_type=sitk_pixel_type,
        big_endian=False,  
        image_spacing= voxel_spacing
    )

    # Save the NIfTI image
    sitk.WriteImage(image, output_path)
    return image  # Return the image for further processing

## Convert to nifti images

In [72]:
from tabulate import tabulate

# Read the CSV file into a DataFrame
csv_file_path =  "4DCT_df.csv"
df = pd.read_csv(csv_file_path)
df = df.sort_values(by='COPD_Type')

table = []

for index, row in df.iterrows():

    # Timer usage
    timer = Timer()

    copd_type = row['COPD_Type']
    _dir = row['folder_path']
    iImage = row['inhale_image']
    eImage = row['exhale_image']

    iLandmark = row['inhale_landmark']
    eLandmark = row['exhale_landmark']

    image_size = row['Dimensions']
    voxel_spacing = row['Voxels']
    
    image_size = ast.literal_eval(image_size)
    voxel_spacing = ast.literal_eval(voxel_spacing)
    
    # inhale raw image convert to nifti
    _raw_path = os.path.join(_dir, iImage)
    iImageName = os.path.splitext(iImage)[0] + ".nii.gz"
    _output_path = os.path.join(_dir, iImageName)

    # Start the timer with a task name
    timer.start(iImage)

    _image = convert_raw_to_nifti(
        raw_file_path=_raw_path,
        image_size=image_size,
        sitk_pixel_type=sitk.sitkInt16,
        output_path=_output_path,
        voxel_spacing=voxel_spacing,
    )
    
    # Stop the timer
    timer.stop()

    # Show total processing time with task name
    inhale_process_time = timer.get_elapsed_time()

    # exhale raw image convert to nifti
    _raw_path = os.path.join(_dir, eImage)
    eImageName = os.path.splitext(eImage)[0] + ".nii.gz"
    _output_path = os.path.join(_dir, eImageName)
     
    # Start the timer with a task name
    timer.start(eImage)

    _image = convert_raw_to_nifti(
        raw_file_path=_raw_path,
        image_size=image_size,
        sitk_pixel_type=sitk.sitkInt16,
        output_path=_output_path,
        voxel_spacing=voxel_spacing,
    )

    # Stop the timer
    timer.stop()

    # Show total processing time with task name
    exhale_process_time = timer.get_elapsed_time()

     # Update the cvt_iImg and cvt_eImg columns
    df.loc[df['COPD_Type'] == copd_type, 'cvt_inhale_image'] = iImageName
    df.loc[df['COPD_Type'] == copd_type, 'cvt_exhale_image'] = eImageName
    df.to_csv("4DCT_df.csv", index=False)

    table.append([
        f"{copd_type}",
        f"{inhale_process_time:.2f}",
        f"{exhale_process_time:.2f}"
    ])


print(tabulate(table, headers=['COPD_Type','inhale Image (s)','exhale Image (s)'], tablefmt="grid"))
print("done!!!")

[b'ObjectType = Image\n', b'NDims = 3\n', b'DimSize = 512 512 121\n', b'ElementSpacing = 0.625 0.625 2.5\n', b'Offset = 0 0 0\n', b'TransformMatrix = 1 0 0 0 1 0 0 0 1\n', b'ElementType = MET_SHORT\n', b'BinaryData = True\n', b'BinaryDataByteOrderMSB = False\n', b'ElementDataFile = /Users/taiaburrahman/Desktop/Udg/MIRA/FinalProject/dataset/train/copd1/copd1_iBHCT.img\n']
[b'ObjectType = Image\n', b'NDims = 3\n', b'DimSize = 512 512 121\n', b'ElementSpacing = 0.625 0.625 2.5\n', b'Offset = 0 0 0\n', b'TransformMatrix = 1 0 0 0 1 0 0 0 1\n', b'ElementType = MET_SHORT\n', b'BinaryData = True\n', b'BinaryDataByteOrderMSB = False\n', b'ElementDataFile = /Users/taiaburrahman/Desktop/Udg/MIRA/FinalProject/dataset/train/copd1/copd1_eBHCT.img\n']
[b'ObjectType = Image\n', b'NDims = 3\n', b'DimSize = 512 512 102\n', b'ElementSpacing = 0.645 0.645 2.5\n', b'Offset = 0 0 0\n', b'TransformMatrix = 1 0 0 0 1 0 0 0 1\n', b'ElementType = MET_SHORT\n', b'BinaryData = True\n', b'BinaryDataByteOrderMSB =